# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Kafka Producer: Financial Transaction Generator** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Team members**: 
- Miguel Alberto Torres Dueñas
- Juan Pablo Cortez Navarro
- Luther Williams Sandria 
- Ferdinand Bierbaum

In [35]:
import findspark
findspark.init()
#0be7b65b50a239d7ee8b621f3c329b25c5c4aadafbae5ac7

In [36]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://f5db43ce3d38:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

In [22]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "67261d1016d7:9093") \
                .option("subscribe", "kafka-spark-example-0") \
                .option("subscribe", "kafka-spark-example-1") \
                .option("subscribe", "kafka-spark-example-2") \
                .option("subscribe", "kafka-spark-example-3") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [37]:
from pyspark.sql.functions import split, col, expr

kafka_df = kafka_lines.select(split(col("value"), ",").alias("pairs_array"))

kafka_df = kafka_df.withColumn("user_id", split(col("pairs_array").getItem(0), ":").getItem(1))
kafka_df = kafka_df.withColumn("video_id", split(col("pairs_array").getItem(1), ":").getItem(1))
kafka_df = kafka_df.withColumn("watch_time_seconds", split(col("pairs_array").getItem(2), ":").getItem(1))
kafka_df = kafka_df.withColumn("resolution", split(col("pairs_array").getItem(3), ":").getItem(1))
kafka_df = kafka_df.withColumn("bitrate_kbps", split(col("pairs_array").getItem(4), ":").getItem(1))
kafka_df = kafka_df.withColumn("buffering_events", split(col("pairs_array").getItem(5), ":").getItem(1))
kafka_df = kafka_df.withColumn("paused", split(col("pairs_array").getItem(6), ":").getItem(1))
kafka_df = kafka_df.withColumn("skipped", split(col("pairs_array").getItem(7), ":").getItem(1))
kafka_df = kafka_df.withColumn("genre", split(col("pairs_array").getItem(8), ":").getItem(1))
kafka_df = kafka_df.withColumn("region", split(col("pairs_array").getItem(9), ":").getItem(1))
kafka_df = kafka_df.withColumn("recommended", split(col("pairs_array").getItem(10), ":").getItem(1))

# Usamos expr para hacer la resta de longitud directamente
kafka_df = kafka_df.withColumn(
    "timestamp",
    expr("substring(split(pairs_array[11], ':')[1], 1, length(split(pairs_array[11], ':')[1]) - 1)")
)

kafka_df.printSchema()


root
 |-- pairs_array: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- user_id: string (nullable = true)
 |-- video_id: string (nullable = true)
 |-- watch_time_seconds: string (nullable = true)
 |-- resolution: string (nullable = true)
 |-- bitrate_kbps: string (nullable = true)
 |-- buffering_events: string (nullable = true)
 |-- paused: string (nullable = true)
 |-- skipped: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- region: string (nullable = true)
 |-- recommended: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [ ]:
# query = kafka_df \
#                 .writeStream \
#                 .outputMode("append") \
#                 .trigger(processingTime='3 seconds') \
#                 .format("console") \
#                 .option("truncate", "false") \
#                 .start()

# query.awaitTermination(300)
# query.stop()

In [28]:
query_files = kafka_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='3 seconds') \
                .format("parquet") \
                .option("path", "/home/jovyan/notebooks/data/project_parquet") \
                .option("truncate", "false") \
                .option("checkpointLocation", "/home/jovyan/checkpoint") \
                .start()
query_files.awaitTermination(300)
query_files.stop()

25/05/12 00:13:34 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/12 00:13:35 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
25/05/12 00:16:24 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 3005 milliseconds
25/05/12 00:16:36 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 3280 milliseconds
25/05/12 00:16:39 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 3092 milliseconds
25/05/12 00:16:48 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 3000 milliseconds, but spent 3641 milliseconds
25/05/12 00:17:00 WARN ProcessingTimeExecutor: Current 

In [38]:
df = spark.read.parquet("/home/jovyan/notebooks/data/project_parquet")
df.show(5, False)


+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------+------------------+----------+------------+----------------+------+-------+--------------+------+-----------+-------------+
|pairs_array                                                                                                                                                                                                                                                                           |user_id     |video_id  |watch_time_seconds|resolution|bitrate_kbps|buffering_events|paused|skipped|genre         |region|recommended|timestamp    |
+-----------------------------------------------------------------------------------------------------------------------------------------------

In [47]:
from pyspark.sql.functions import col, regexp_extract, when

# Leer datos Parquet
video_data = spark.read.parquet("/home/jovyan/notebooks/data/project_parquet")

# Convertir tipos de datos correctamente
video_data = video_data.withColumn("watch_time_seconds", col("watch_time_seconds").cast("double")) \
                      .withColumn("buffering_events", col("buffering_events").cast("double")) \
                      .withColumn("skipped", when(col("skipped") == "true", 1).otherwise(0))

# Crear IDs numéricos (como lo tienes)
video_data = video_data.withColumn("user_id_int", regexp_extract(col("user_id"), "user_(\\d+)", 1).cast("integer")) \
                      .withColumn("video_id_int", regexp_extract(col("video_id"), "vid_(\\d+)", 1).cast("integer"))

# Crear métrica de feedback implícito (versión corregida)
video_data = video_data.withColumn("implicit_feedback", 
                                 (col("watch_time_seconds")/3600 * 0.6 +  # Tiempo de visualización en horas
                                  (1 - col("buffering_events")/10) * 0.3 +  # Inverso de buffering
                                  (1 - col("skipped")) * 0.1))  # No saltado

# Verificar datos
video_data.select("user_id_int", "video_id_int", "implicit_feedback").show(5)

+-----------+------------+------------------+
|user_id_int|video_id_int| implicit_feedback|
+-----------+------------+------------------+
|       4137|          34|0.9181666666666666|
|       9570|          13|0.4986666666666667|
|       6444|          76|0.6296666666666667|
|       9572|          78|0.4571666666666666|
|       5070|          87|0.8571666666666666|
+-----------+------------+------------------+
only showing top 5 rows



In [48]:
from pyspark.ml.recommendation import ALS

als = ALS(
    userCol="user_id_int",
    itemCol="video_id_int",
    ratingCol="implicit_feedback",
    implicitPrefs=True,  # Usar feedback implícito
    coldStartStrategy="drop",  # Manejar nuevos usuarios/videos
    nonnegative=True,  # Solo factores positivos
    rank=10,  # Factores latentes (ajustable)
    maxIter=15,  # Iteraciones (ajustable)
    regParam=0.1  # Regularización (ajustable)
)

In [ ]:
# Entrenar modelo
model = als.fit(video_data)

In [59]:
# Recomendaciones para todos los usuarios (top 10)
user_recs = model.recommendForAllUsers(5)

# Mostrar recomendaciones para algunos usuarios
print("Recomendaciones para usuarios de ejemplo:")
user_recs.show(truncate=False)

# Recomendaciones para un usuario específico
user_id_ejemplo = 4137
recs_usuario = model.recommendForUserSubset(
    spark.createDataFrame([(user_id_ejemplo,)]).toDF("user_id_int"), 
    5
)
print(f"\nTop 5 recomendaciones para usuario {user_id_ejemplo}:")
recs_usuario.show(truncate=False)

Recomendaciones para usuarios de ejemplo:


+-----------+---------------------------------------------------------------------------------------------------+
|user_id_int|recommendations                                                                                    |
+-----------+---------------------------------------------------------------------------------------------------+
|1005       |[{42, 0.12646593}, {74, 0.111349694}, {34, 0.10814677}, {58, 0.09765415}, {83, 0.040566683}]       |
|1008       |[{38, 2.715734E-8}, {54, 2.3733621E-8}, {24, 1.9056523E-8}, {76, 5.8706338E-9}, {70, 5.5232894E-9}]|
|1063       |[{65, 0.89899695}, {99, 0.28704473}, {23, 0.040367953}, {26, 0.008795763}, {48, 0.0054560807}]     |
|1069       |[{42, 0.38380265}, {74, 0.33785674}, {34, 0.32813844}, {58, 0.29630178}, {83, 0.1227222}]          |
|1070       |[{36, 0.37224424}, {97, 0.36538333}, {16, 0.33246157}, {1, 0.20937836}, {71, 0.16913295}]          |
|1154       |[{73, 0.79032665}, {75, 0.39243704}, {97, 0.049660176}, {78, 7.2148384E-4},

+-----------+--------------------------------------------------------------------------------------+
|user_id_int|recommendations                                                                       |
+-----------+--------------------------------------------------------------------------------------+
|4137       |[{42, 0.40088734}, {74, 0.35297}, {34, 0.34281698}, {58, 0.30955616}, {83, 0.1282119}]|
+-----------+--------------------------------------------------------------------------------------+



In [60]:
from pyspark.ml.evaluation import RegressionEvaluator

# Hacer predicciones
predictions = model.transform(video_data)
predictions.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+----------+------------------+----------+------------+----------------+------+-------+--------------+------+-----------+-------------+-----------+------------+-------------------+-------------+
|pairs_array                                                                                                                                                                                                                                                                           |user_id     |video_id  |watch_time_seconds|resolution|bitrate_kbps|buffering_events|paused|skipped|genre         |region|recommended|timestamp    |user_id_int|video_id_int|implicit_feedback  |prediction   |
+-------------------------

In [61]:
# Evaluar con RMSE
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="implicit_feedback",
    predictionCol="prediction"
)
rmse = evaluator.evaluate(predictions)
print(f"\nRMSE del modelo: {rmse}")


RMSE del modelo: 0.4791046338139269


In [34]:
sc.stop()